In [ ]:

!pip install boto3 sagemaker


In [3]:
import sagemaker
from sagemaker.pytorch import PyTorchModel
import boto3
import datetime
import time
from time import strftime,gmtime
import json
import os
import urllib
import sys
import io

boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto_session.client("sagemaker-runtime")
sns_client = boto3.client('sns')
region = boto_session.region_name
bucket="lightsketch-models-188775091215"
prefix = 'async-inference'

print(region)
print(prefix)

us-east-1
async-inference


### Delete Role

In [4]:
import boto3
import json
from botocore.exceptions import ClientError

# Specify the role name
role_name = 'SageMaker-Role'

# Create an IAM client
iam_client = boto3.client('iam')

try:
    response = iam_client.list_attached_role_policies(RoleName=role_name)
    attached_policies = response['AttachedPolicies']
    # Detach policies
    for policy in attached_policies:
        policy_arn = policy['PolicyArn']
        iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
        print(f"Detached policy: {policy_arn}")

# Delete inline policies
    iam_client.delete_role(RoleName=role_name)
except Exception as e:
    print(e)

Detached policy: arn:aws:iam::aws:policy/AmazonSNSFullAccess
Detached policy: arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
Detached policy: arn:aws:iam::aws:policy/AmazonS3FullAccess


### Create Role

In [5]:
import boto3
import json
from botocore.exceptions import ClientError

# Specify the role name
role_name = 'SageMaker-Role'

# Managed policies for SageMaker
managed_policy_arns = [
    'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
    'arn:aws:iam::aws:policy/AmazonS3FullAccess',
    'arn:aws:iam::aws:policy/AmazonSNSFullAccess'
]

# Create an IAM client
iam_client = boto3.client('iam')
    
# Create the role
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "sagemaker.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
    )
    print("SageMaker role created successfully:", create_role_response['Role']['Arn'])
    
    # Attach managed policies to the role
    for policy_arn in managed_policy_arns:
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )
        print(f"Attached policy {policy_arn} to the role.")

except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role with the same name already exists.")
    else:
        print("Error creating SageMaker role:", e)


SageMaker role created successfully: arn:aws:iam::188775091215:role/SageMaker-Role
Attached policy arn:aws:iam::aws:policy/AmazonSageMakerFullAccess to the role.
Attached policy arn:aws:iam::aws:policy/AmazonS3FullAccess to the role.
Attached policy arn:aws:iam::aws:policy/AmazonSNSFullAccess to the role.


In [6]:
role_arn = create_role_response['Role']['Arn']

In [7]:
role_arn="arn:aws:iam::188775091215:role/SageMaker-Role"

In [8]:
from sagemaker.pytorch import PyTorchModel
# Import packages
import boto3 # AWS Session Management
import sagemaker # SageMaker SDK

model = PyTorchModel(entry_point='inference.py',
                     model_data="s3://lightsketch-models-188775091215/models/model.tar.gz",
                     framework_version='1.12',
                     py_version='py38',
                     role=role_arn
                     )

In [9]:
response = sns_client.create_topic(Name="Async-ML-ErrorTopic")
error_topic= response['TopicArn']
print(error_topic)

arn:aws:sns:us-east-1:188775091215:Async-ML-ErrorTopic


In [10]:
response = sns_client.create_topic(Name="Async-ML-SuccessTopic")
success_topic = response['TopicArn']
print(success_topic)

arn:aws:sns:us-east-1:188775091215:Async-ML-SuccessTopic


In [11]:
response = sns_client.list_topics()
topics = response["Topics"]
print(topics)

[{'TopicArn': 'arn:aws:sns:us-east-1:188775091215:Async-ML-ErrorTopic'}, {'TopicArn': 'arn:aws:sns:us-east-1:188775091215:Async-ML-SuccessTopic'}, {'TopicArn': 'arn:aws:sns:us-east-1:188775091215:PipelineApproval'}]


In [12]:


email_id = 'mfahadm8@gmail.com'
email_sub_1 = sns_client.subscribe(
    TopicArn=success_topic,
    Protocol='email',
    Endpoint=email_id)

email_sub_2 = sns_client.subscribe(
    TopicArn=error_topic,
    Protocol='email',
    Endpoint=email_id)


## Simplified Way

### Realtime

In [ ]:
from sagemaker.deserializers import JSONDeserializer
from datetime import datetime
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig

INSTANCE_TYPE = 'ml.c5.xlarge'
ENDPOINT_NAME = 'yolov7-rt-' + str(datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f'))

predictor = model.deploy(initial_instance_count=1,
                         instance_type=INSTANCE_TYPE,
                         deserializer=JSONDeserializer(),
                         endpoint_name=ENDPOINT_NAME,
                         env={
                            'TS_MAX_REQUEST_SIZE': '100000000',
                            'TS_MAX_RESPONSE_SIZE': '100000000',
                            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'  
                         }
                         )

#### realtime testing

In [ ]:
feed_data = open('temp/20200616_VB_trim.mp4', 'rb')
sm_runtime = boto3.Session().client('sagemaker-runtime',region_name="us-east-1")
r = sm_runtime.invoke_endpoint(EndpointName="realtime", Body=feed_data)

## Async Endpoint Creation Step By Step

In [14]:
from sagemaker.image_uris import retrieve

deploy_instance_type = 'ml.c5.xlarge'
pytorch_inference_image_uri = retrieve('pytorch',
                                       region,
                                       version='1.12',
                                       py_version='py38',
                                       instance_type = deploy_instance_type,
                                       accelerator_type=None,
                                       image_scope='inference')
print(pytorch_inference_image_uri)

763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.12-cpu-py38


In [15]:
container = pytorch_inference_image_uri
model_artifact="s3://lightsketch-models-188775091215/models/model.tar.gz"
model_name = 'ball-tracking-yolo7-{0}'.format(str(int(time.time())))
print(container)
print(model_name)

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role_arn,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_artifact,
        'Mode': 'SingleModel',
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL':'20',
            'SAGEMAKER_PROGRAM':'inference.py',
            'SAGEMAKER_REGION':region,
            'SAGEMAKER_SUBMIT_DIRECTORY':'/opt/ml/model/code',
            'TS_MAX_REQUEST_SIZE': '100000000', #default max request size is 6 Mb for torchserve, need to update it to support the 70 mb input payload
            'TS_MAX_RESPONSE_SIZE': '100000000',
            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'
        }
    },    
)

763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.12-cpu-py38
ball-tracking-yolo7-1692830551


In [19]:
print(model_name)
endpoint_config_name = f"PyTorchAsyncEndpointConfig-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.c5.xlarge",
            "InitialInstanceCount": 1
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": f"s3://{bucket}/{prefix}/output",
            #  Optionally specify Amazon SNS topics
            "NotificationConfig": {
              "SuccessTopic": success_topic,
              "ErrorTopic": error_topic,
            }
        },
        "ClientConfig": {
            "MaxConcurrentInvocationsPerInstance": 2
        }
    }
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

ball-tracking-yolo7-1692830551
Created EndpointConfig: arn:aws:sagemaker:us-east-1:188775091215:endpoint-config/pytorchasyncendpointconfig-2023-08-23-22-47-09


In [20]:
endpoint_name = f"sm-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
create_endpoint_response = sm_client.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)
print(f"Creating Endpoint: {create_endpoint_response['EndpointArn']}")

Creating Endpoint: arn:aws:sagemaker:us-east-1:188775091215:endpoint/sm-2023-08-23-22-47-14


In [21]:
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

Waiting for endpoint to create...
Endpoint Status: InService


### Testing

In [ ]:
response = sm_runtime.invoke_endpoint_async(
    EndpointName="test", 
    InputLocation="s3://lightsketch-models-188775091215/models/20200616_VB_trim.mp4")
output_location = response['OutputLocation']
print(f"OutputLocation: {output_location}")

In [ ]:
print(model_name)
endpoint_config_name = f"PyTorchAsyncEndpointConfig-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.c5.xlarge",
            "InitialInstanceCount": 1,
            
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": f"s3://{bucket}/{prefix}/output",
            #  Optionally specify Amazon SNS topics
            "NotificationConfig": {
              "SuccessTopic": success_topic,
              "ErrorTopic": error_topic,
            }
        },
        "ClientConfig": {
            "MaxConcurrentInvocationsPerInstance": 2
        }
    }
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

In [ ]:
endpoint_name = f"sm-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
create_endpoint_response = sm_client.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)
print(f"Creating Endpoint: {create_endpoint_response['EndpointArn']}")

In [ ]:
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

### Invoke asynchronous endpoint

In [ ]:
response = sm_runtime.invoke_endpoint_async(
    EndpointName=endpoint_name, 
    InputLocation=input_1_s3_location)
output_location = response['OutputLocation']
print(f"OutputLocation: {output_location}")

In [ ]:
from botocore.exceptions import ClientError

def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    while True:
        try:
            return sm_session.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
        except ClientError as e:
            if e.response['Error']['Code'] == 'NoSuchKey':
                print("waiting for output...")
                time.sleep(2)
                continue
            raise


In [ ]:
output = get_output(output_location)
print(f"Output size in bytes: {((sys.getsizeof(output)))}")